In [1]:
import os
import time
from fnmatch import fnmatch
import json
import csv
import pickle
import random
import pandas as pd
import numpy as np
import requests
import tweepy
import pprint

In [28]:
def bearer():
    return os.getenv('TOKEN')

In [2]:
paths = []
root = "../russo_ukraine_dataset"
pattern = pattern = "*.csv"

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            paths.append(os.path.join(path, name))
            #print(os.path.join(path, name))
            
print(len(paths))

196


In [207]:
n_files = 100
n_tweets = 10

sample_tweet_ids = []

for path in random.sample(paths, n_files):
    with open(path, "r") as file:
        tweet_ids = [tweet_id.strip() for tweet_id in file.readlines()]
        sample_tweet_ids += random.sample(tweet_ids, n_tweets)
        
pprint.pprint(sample_tweet_ids)

['1501715377335775235',
 '1501734364664590338',
 '1501715719205097472',
 '1501725096087588866',
 '1501734648711245827',
 '1501725135966924801',
 '1501724938885124098',
 '1501724894052204551',
 '1501725058053586948',
 '1501725114643238917',
 '1506721233156591627',
 '1506721089086447624',
 '1506759271027036167',
 '1506773093343436803',
 '1506750542571257868',
 '1506711270535356429',
 '1506782626215956483',
 '1506759253691932673',
 '1506731462028111879',
 '1506731209673564168',
 '1498963597845803008',
 '1499001782583889924',
 '1499010549522354179',
 '1498963565071716354',
 '1498984531935645697',
 '1498984645244993543',
 '1498975882697969666',
 '1498955016987897859',
 '1498955096037543937',
 '1498963698395852807',
 '1505280958773317637',
 '1505281024770641926',
 '1505271659212816391',
 '1505280893681815561',
 '1505262112431656967',
 '1505299698181156870',
 '1505280854829936643',
 '1505242509223636995',
 '1505233572390739971',
 '1505252801965965317',
 '1506217537951997953',
 '15062044839986

### API 1 (Old version!)

(I was testing this because I originally had some problems with the authentication for API 2, but it works now - scroll down!)

In [29]:
auth = tweepy.OAuth2BearerHandler(bearer())
api = tweepy.API(auth)

In [54]:
sleep_time = 2
for tweet_id in sample_tweet_ids:
    print("---")
    try:
        tweet = api.get_status(tweet_id)
        print(tweet.text)
    except Exception as e:
        print(e)
    time.sleep(sleep_time)

---
RT @annabosch: Podemos pensar y argumentar que nosotros, #Occidente, no estamos en #Guerra con #Rusia, pero #Putin volvió a insistir ayer e…
---
RT @tcbestepe: Cumhurbaşkanı @RTErdogan, Rusya Devlet Başkanı Vladimir Putin ile bir telefon görüşmesi gerçekleştirdi. 

Görüşmede, Rusya’n…
---
we're not making sacrifices to the great god of inflation, to support ukraine over russia, we're making these sacri… https://t.co/3fmnNPZHPm
---
RT @DeanStoltzfus: Explain to me like I’m 5 why drilling for oil in Russia or Iran or anywhere else doesn’t harm the environment? Only in N…


### API 2 (Correct version!)

In [171]:
client = tweepy.Client(bearer_token=bearer(), wait_on_rate_limit = True)

In [262]:
all_tweets = {}

for i, tweet_id in enumerate(sample_tweet_ids):

    tweet = client.get_tweet(tweet_id,
                                tweet_fields=['context_annotations','created_at','geo'],
                                user_fields=['profile_image_url'], 
                                expansions=['author_id', 'geo.place_id', 'referenced_tweets.id'],
                                place_fields = ["contained_within", "country", "country_code", "full_name", 
                                                "geo", "id", "name", "place_type"]
                               )
    if tweet.data:
        all_tweets[tweet_id] = tweet.data.data
    done = 100*i/len(sample_tweet_ids)
    print(int(done)*"|"+(100-int(done))*"_" + "\t"+str(round(done,2))+"%", end = "\r")
    
print()
print("100%")

Rate limit exceeded. Sleeping for 276 seconds.


Rate limit exceeded. Sleeping for 841 seconds.


Rate limit exceeded. Sleeping for 839 seconds.


Rate limit exceeded. Sleeping for 838 seconds.


|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||_	99.9%
100%


In [276]:
with open('tweets_1000.json', 'w') as fp:
    json.dump(all_tweets, fp)

In [4]:
with open('tweets_1000.json', 'r') as file:
    all_tweets = json.load(file)

In [5]:
for tweet in all_tweets:
    tweet_dict = all_tweets[tweet]
    if not "referenced_tweets" in tweet_dict.keys():
        pprint.pprint(tweet_dict)
        print()

{'author_id': '1490822650905694213',
 'context_annotations': [{'domain': {'description': 'A section of a '
                                                    'government, like The '
                                                    'Supreme Court',
                                     'id': '88',
                                     'name': 'Political Body'},
                          'entity': {'description': 'North Atlantic Treaty '
                                                    'Organization',
                                     'id': '1016738021939351558',
                                     'name': 'North Atlantic Treaty '
                                             'Organization'}}],
 'created_at': '2022-03-23T22:26:19.000Z',
 'id': '1506759271027036167',
 'text': 'As many as 15,000 Russian troops have been killed in Ukraine, NATO '
         'says: https://t.co/toL9Ar0bQV. Not accord’g to Carlson. He updated '
         'his previous numbers from 100 soldiers &amp; 0 Ge

                                     'name': 'COVID-19'}},
                         {'domain': {'description': 'Named people in the world '
                                                    'like Nelson Mandela',
                                     'id': '10',
                                     'name': 'Person'},
                          'entity': {'description': 'Joy Reid',
                                     'id': '872880012436717568',
                                     'name': 'Joy Reid'}},
                         {'domain': {'description': 'A journalist like '
                                                    "'Anderson Cooper'",
                                     'id': '94',
                                     'name': 'Journalist'},
                          'entity': {'description': 'Joy Reid',
                                     'id': '872880012436717568',
                                     'name': 'Joy Reid'}}],
 'created_at': '2022-03-10T03:32:31.000Z',
 'id'

In [259]:
import math

def get_type(refs):
    
    if not isinstance(refs, list):
        return "tweet"
    else:
        return refs[0]["type"]

In [264]:
df_all_tweets = pd.DataFrame.from_dict(all_tweets)
df_all_tweets = df_all_tweets.transpose().set_index("id")
df_all_tweets["type"] = df_all_tweets.apply(lambda row: get_type(row["referenced_tweets"]), axis=1)
df_all_tweets

,text,author_id,created_at,referenced_tweets,context_annotations,withheld,geo,type
id,,,,,,,,
1501715377335775235,RT @stillgray: U.S. Under Secretary of State V...,115792183,2022-03-10T00:23:41.000Z,"[{'type': 'retweeted', 'id': '1501310103529541...",NaN,NaN,NaN,retweeted
1501734364664590338,@BHDonk1 Russia took Crimea after the U.S. fun...,1376636130246860801,2022-03-10T01:39:08.000Z,"[{'type': 'replied_to', 'id': '150172868701149...",NaN,NaN,NaN,replied_to
1501715719205097472,RT @anneapplebaum: There is always a military ...,285005616,2022-03-10T00:25:02.000Z,"[{'type': 'retweeted', 'id': '1501700149332090...",NaN,NaN,NaN,retweeted
1501725096087588866,"RT @RonFilipkowski: It’s Guilfoyle’s birthday,...",1948262269,2022-03-10T01:02:18.000Z,"[{'type': 'retweeted', 'id': '1501706894905585...","[{'domain': {'id': '10', 'name': 'Person', 'de...",NaN,NaN,retweeted
1501734648711245827,RT @EpicSerbian: Russians pre-empted Donbass a...,257097577,2022-03-10T01:40:15.000Z,"[{'type': 'retweeted', 'id': '1501580337993838...",NaN,NaN,NaN,retweeted
...,...,...,...,...,...,...,...,...
1498075055573676033,"RT @ataraxie66: Etrange quand même, on trouve ...",1135125324,2022-02-27T23:18:20.000Z,"[{'type': 'retweeted', 'id': '1498071752345276...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",NaN,NaN,retweeted
1498075067812614144,@RadioFreeTom Maybe Putin is going through an ...,3798288154,2022-02-27T23:18:23.000Z,"[{'type': 'replied_to', 'id': '149804628286630...","[{'domain': {'id': '10', 'name': 'Person', 'de...",NaN,NaN,replied_to
1498074904461205504,"@Judmir3 Its fake, I don't know what this is b...",1370452383889772546,2022-02-27T23:17:44.000Z,"[{'type': 'replied_to', 'id': '149805059902325...",NaN,NaN,NaN,replied_to


In [282]:
df_only_tweets = df_all_tweets[df_all_tweets["type"] == "tweet"]
df_only_tweets

,text,author_id,created_at,referenced_tweets,context_annotations,withheld,geo,type
id,,,,,,,,
1506759271027036167,"As many as 15,000 Russian troops have been kil...",1490822650905694213,2022-03-23T22:26:19.000Z,NaN,"[{'domain': {'id': '88', 'name': 'Political Bo...",NaN,NaN,tweet
1506711270535356429,"Ronan Calvarin, de Ploudalmézeau, ne renonce p...",2315286290,2022-03-23T19:15:34.000Z,NaN,NaN,NaN,NaN,tweet
1505280958773317637,"Ukraine : Révélations explosives, à écouter ju...",1375927250529452032,2022-03-19T20:32:01.000Z,NaN,NaN,NaN,NaN,tweet
1506196506595975169,Horrible what happened to @navalny today. We a...,1209233484616781834,2022-03-22T09:10:05.000Z,NaN,"[{'domain': {'id': '10', 'name': 'Person', 'de...",NaN,NaN,tweet
1506151734145871874,Ukraine : Zelensky prêt à discuter du Donbass ...,34570323,2022-03-22T06:12:11.000Z,NaN,"[{'domain': {'id': '10', 'name': 'Person', 'de...",NaN,NaN,tweet
...,...,...,...,...,...,...,...,...
1509770950086119432,‘My Camera Is My Shield’: A Ukrainian Photojou...,259329309,2022-04-01T05:53:39.000Z,NaN,NaN,NaN,NaN,tweet
1508438909633630221,"putin accetta solo pagamenti in rubli, o nient...",1349480839969648645,2022-03-28T13:40:36.000Z,NaN,"[{'domain': {'id': '10', 'name': 'Person', 'de...",NaN,{'place_id': 'a6df3afa975fad24'},tweet
1508438577578921996,"Guerre Ukraine : qu'est ce que le Javelin, cet...",1358114448054099969,2022-03-28T13:39:16.000Z,NaN,"[{'domain': {'id': '10', 'name': 'Person', 'de...",NaN,NaN,tweet
